Lineare Regression
---
Als ersten Anwendungsfall wird ein simples Maschinellen Lernen Verfahren betrachtet, die Lineare Regression. Hierbei wird ein mathematisches Modell durch Trainingsdaten erstellt, welches verwendet werden kann um ein ausgewähltes Feature vorherzusagen. Ein überschaubarer Datensatz für diese Aufgabe wäre zum Beispiel der Iris Datensatz. Dieser beinhaltet Features von drei verschiedenen Blumenarten, welche wiederum durch eine lineare Regression bestimmt werden können.
<br />
Zunächst muss dafür der Datensatz geladen und in die Trainings- und Testmenge aufgeteilt werden. Die beiden Mengen stehen dabei in einem 80% Trainingsdaten - 20% Testdaten Verhältnis.

In [1]:
# Import Dataset from sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

Bibliotheken Beispiel
---
Um einen Eindruck für eine Lineare Regression auf dem Iris Datensatz zu erhalten wird zuerst die Sklearn Implementierung ausprobiert. Die berechneten Abweichungen von den Ergebnissen können wiederum verglichen werden mit weiteren Tests.

In [2]:
lr = LinearRegression()

lr.fit(X_train, y_train)

lr.predict(X_test)
pred = lr.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squared Error:', mean_squared_error(y_test, pred))
print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

Mean Absolute Error: 0.1584601230068247
Mean Squared Error: 0.043573599004634006
Mean Root Squared Error: 0.20874290168682144


From Scratch
---
Da die Implementierung aus Sklearn nicht jeden Datentyp unterstützt muss die Lineare Regression selbst implementiert werden. Der Grund dafür liegt in der homomorphen Verschlüsselung. Sobald eine beliebige Zahl verschlüsselt wurde, ist diese keine gewöhnliche Zahl mehr sonder ein verschlüsseltes Objekt und die meisten Bibliotheken erlauben als Parameter nur Zahlen beziehungsweise primitive Datentypen.
<br />
Für die Lineare Regression werden zwei Funktionen implementiert:
* Fit: In dieser Funktion werden die Gewichte w in der Lineare Regression basierend auf den Trainingsdaten erstellt. Dies geschieht indem immer wieder mit den Gewichten Vorhersagen erstellt werden und verändert werden bis diese konvergieren.
* Predict: In dieser Funktion wird die Vorhersage mit einer beliebigen Eingabe und den in der Predict Funktion berechneten Gewichte erzeugt.

In [3]:
def fit(x, y):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])

	norma = 1
	learning_rate = 0.00001
	epsilon = 0.9
	count = 0
	while(norma > epsilon):
		count = count + 1
		y_pred = regression @ weights.T
		dw = regression.T @ (y - y_pred)
		norma = np.sum(np.sqrt(np.square(dw)))

		weights = weights.T + (learning_rate * dw)
	print(count)
	return weights

def predict(w, x):
	return w[:-1] @ (np.array(x).T) + w[-1]

Nachdem die Funktionen implementiert sind können die Gewichte erstellt werden mit den Trainingsdaten. Sobald diese berechnet sind wird ein Feature von den Testdaten vorhersagt. Dabei wird noch die Laufzeit der Vorhersage gemessen um später mit den anderen Tests verglichen zu werden.
<br />
Die erstellten Gewichte können in den Tests mit der homomorphen Verschlüsselung wieder verwendet werden.

In [4]:
weights = fit(X_train, y_train)

s_predict = datetime.datetime.now()
pred = predict(weights, X_test)
e_predict = datetime.datetime.now()

def meanError(y_test, pred):
	print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
	print('Mean Squared Error:', mean_squared_error(y_test, pred))
	print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

meanError(y_test, pred)

print(f"Prediction time for original: {int((e_predict-s_predict).total_seconds() * 1000)}ms")

def timer(encryptTime, predictTime, decryptTime):
	print(f"Encryption time: {int(encryptTime.total_seconds() * 1000)}ms")
	print(f"Prediction time: {int(predictTime.total_seconds() * 1000)}ms")
	print(f"Decryption time: {int(decryptTime.total_seconds() * 1000)}ms")


7470
Mean Absolute Error: 0.15791782248674335
Mean Squared Error: 0.042219357807520355
Mean Root Squared Error: 0.2054734966060595
Prediction time for original: 0ms


Die berechneten Abweichung zwischen den Vorhersagen und den tatsächlichen Features sind ähnlich zu den Ergebnissen der Sklearn Implementierung und weisen erst ab der 3. Nachkommastelle Unterschiede auf. Die eigene Implementierung scheint dadurch korrekt zu funktionieren.

Vorhersagen auf verschlüsselten Daten
===
In den folgenden zwei Beispielen werden die mit unverschlüsselten Daten trainierten Gewichte angewendet auf verschlüsselte Daten.

Verschlüsselung mit Pyfhel
---

In [5]:
from Pyfhel import Pyfhel

pyfhel = Pyfhel()
pyfhel.contextGen(p=65537)

pyfhel.keyGen()

def encrypt(data):
    return [[pyfhel.encryptFrac(feature) for feature in item] for item in data]

def decrypt(data):
    return [pyfhel.decryptFrac(item) for item in data]

Für diese Bibliothek muss die predict Funktion etwas umgeschrieben werden, da die Matrizenmultiplikation Operation nicht funktioniert zwischen Skalaren und verschlüsselten Objekten.

In [6]:
def predictPyfhel(w, x):
    transX = np.array(x).T
    result = []
    for i in range(transX.shape[1]):
        for j in range(transX.shape[0]):
            scaledFeature = transX[j][i] * w[j]
            if len(result) == i:
                result.append(scaledFeature)
            else:
                result[i] = result[i] + scaledFeature
    return np.array(result) + w[-1]

Als letzter Schritt werden alle Testdaten verschlüsselt, die Vorhersage erzeugt und das Ergebnis entschlüsselt.

In [7]:
s_encrypt = datetime.datetime.now()
eX_test = encrypt(X_test)
e_encrypt = datetime.datetime.now()

s_predict = datetime.datetime.now()
ePred = predictPyfhel(weights, eX_test)
e_predict = datetime.datetime.now()

s_decrypt = datetime.datetime.now()
cryptPred = decrypt(ePred)
e_decrypt = datetime.datetime.now()

meanError(y_test, cryptPred)
timer(e_encrypt-s_encrypt, e_predict-s_predict, e_decrypt-s_decrypt)

Mean Absolute Error: 0.1579178223464017
Mean Squared Error: 0.04221935761486088
Mean Root Squared Error: 0.20547349613724122
Encryption time: 121ms
Prediction time: 19ms
Decryption time: 4ms


* Funktionalität: Ergebnisse sind fast identisch zum Original (Abweichungen in der Gleitkommarepräsentation)
* Laufzeit: Fast gleich schnell wie die Implementierung

Verschlüsselung mit Paillier
---

In [8]:
from phe import paillier

public_key, private_key = paillier.generate_paillier_keypair()

def encrypt(data):
    return [[public_key.encrypt(feature) for feature in item] for item in data]

def decrypt(data):
    return [private_key.decrypt(item) for item in data]

Für die Paillier Bibliothek in Python muss keine gesonderte predict Funktion implementiert werden, da die Matrizenmultiplikation möglich ist. Dadurch müssen nur noch die Daten verschlüsselt, die Vorhersage erzeugt und das Ergebnis entschlüsselt werden.

In [9]:
s_encrypt = datetime.datetime.now()
eX_test = encrypt(X_test)
e_encrypt = datetime.datetime.now()

s_predict = datetime.datetime.now()
ePred = predict(weights, eX_test)
e_predict = datetime.datetime.now()

s_decrypt = datetime.datetime.now()
cryptPred = decrypt(ePred)
e_decrypt = datetime.datetime.now()

meanError(y_test, cryptPred)
timer(e_encrypt-s_encrypt, e_predict-s_predict, e_decrypt-s_decrypt)

Mean Absolute Error: 0.15791782248674335
Mean Squared Error: 0.042219357807520355
Mean Root Squared Error: 0.2054734966060595
Encryption time: 9872ms
Prediction time: 518ms
Decryption time: 696ms


* Funktionalität: Identisches Ergebnis zu der unverschlüsselten Version
* Laufzeit: Benötigt deutlich länger

Training auf verschlüsselten Daten
===
Für das Training auf verschlüsselten Daten müssen auch die Gewichte der Linearer Regression verschlüsselt werden. Nachdem Training können die Gewichte wieder entschlüsselt und wieder für die Inferenz genutzt werden. Hierfür verschlüsselt der Datenbesitzer seine Daten und verschickt diese zusammen mit dem öffentlichen Schlüssel.

In [66]:
import numpy as np
import tenseal as ts

poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

def encryptX(data, context):
    return [[ts.ckks_vector(context, [feature]) for feature in item] for item in data]

def encryptY(data, context):
    return [ts.ckks_vector(context, [label]) for label in data]

def decrypt(data):
    return [item.decrypt() for item in data]

seal_x_train = encryptX(X_train,ctx_training)
seal_y_train = encryptY(y_train,ctx_training)

Um das Netz zu trainieren gibt es zwei Möglichkeiten:
* Nach jedem Trainingsschritt können die Gewichte entschlüsselt werden, durch den Besitzer des zugehörigen privaten Schlüssels, um dann wieder verschlüsselt zu werden. Dies würde erlauben die Berechnung so oft wie gewünscht durchzuführen. Das Problem hierbei ist, dass dies ein großer Zeit und Rechenaufwand mit sich bringt.
* Die Gewichte müssen aber nicht nach jedem Trainingsschritt entschlüsselt werden und es kann für eine begrenzte Anzahl von Operationen trainiert werden. Diese Variante verringert aber die erreichbare Genauigkeit der trainierten Gewichte.

Kein Ausstausch mit Datenbesitzer
---

Um den Aufwand für den Datenbesitzer niedrig zu halten wird die zweite Variante implementiert und getestet. Hierfür wird die fit Funktion verändert, da nicht alle Vorgänge durchführbar sind mit verschlüsselten Daten. So können zum Beispiel die Gewichte nicht solange optimiert werden bis diese konvergieren, weshalb es nur wenige Trainingsphasen geben wird. Damit das Training trotzdem erfolgreich ist ist es hielfreich verschiedene Lernraten auszuprobieren, um die Gewichte gut zu approximieren.

In [67]:
hyperparameter = [2, 1, 0.1, 0.01, 0.007, 0.003, 0.002, 0.001, 0.0009, 0.0007, 0.0003, 0.0001, 0.00001]

def fit(x, y, context):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])
	e_weights = np.array(encryptY(weights, context))

	all_weights = []
	for learning_rate in hyperparameter:
		hyper_weights = e_weights
		for i in range(2):
			y_pred = regression @ hyper_weights.T
			dw = regression.T @ (y - y_pred)
			hyper_weights = hyper_weights.T + (learning_rate * dw)

		all_weights.append((learning_rate, hyper_weights))

	return all_weights

Das Training gibt also eine Liste an verschiedenen Gewichten aus für die unterschiedlichen Lernraten.

In [68]:
all_weights = fit(seal_x_train, seal_y_train, ctx_training)

In [69]:
e_weights = [[e_value.decrypt()[0] for e_value in e_tuple[1]] for e_tuple in all_weights]

Die verschiedenen Versionen der Gewichte können nun einzelnen geprüft werden auf dem Testdatensatz. Dadurch kann bestimmt werden welche Lernrate für diese Aufgabe die Beste ist.

In [70]:
min = []
for idx, learning_rate in enumerate(hyperparameter):
    result = predict(e_weights[idx], X_test)
    min.append(np.sqrt(mean_squared_error(y_test, result)))
best_weight_idx = np.argmin(min)
print(f"Beste Gewichte sind die mit der Learning rate von {hyperparameter[best_weight_idx]}")

Beste Gewichte sind die mit der Learning rate von 0.002


Die besten trainierten Gewichte haben eine höhere Abweichung als die Gewichte der unverschlüsselten Modelle. Es kommt aber nahe dran und kann noch weiter verbessert werden mit mehr Trainingsiterationen.

In [71]:
from random import randint

index = randint(0, len(X_test)-1)
print(f"""
Eingabe: {X_test[index]}
Wahres Ergebnis: {y_test[index]}
Ergebnis aus dem verschlüsselt trainierten Modell: {predict(e_weights[best_weight_idx], X_test[index])}
Ergebnis aus dem nicht verschlüsselt trainierten Modell: {predict(weights, X_test[index])}
""")


Eingabe: [ 1.64384411 -0.13197948  1.16062026  0.52740629]
Wahres Ergebnis: 2
Ergebnis aus dem verschlüsselt trainierten Modell: 1.5735141565652482
Ergebnis aus dem nicht verschlüsselt trainierten Modell: 1.5767759592337507



In [72]:
result = predict(e_weights[best_weight_idx], X_test)
meanError(y_test, result)

Mean Absolute Error: 0.31726084294777773
Mean Squared Error: 0.1310889495171555
Mean Root Squared Error: 0.3620620796454049


Der Unterschied im einzelnen Beispiel ist minimal und auch die eizelnen Metriken zeigen auf, dass das mit verschlüsselten Daten trainierte Modell zwar schlechter ist als die vorherig trainierten Modelle aber trotzdem in einem akzeptablen Bereich liegt.

Austausch mit dem Datenbesitzer
---

Die erste Variante leidet unter den begrenzenten Operationen auf den verschlüsselten Gewichten. Vor allem wird dabei die Genauigkeit zugunsten der Laufzeit geopfert. In der zweiten Variante werden die Gewichte nach jeder Veränderung an den Datenbesitzer geschickt, damit diese entschlüsselt und neu verschlüsselt werden können. Damit ist eine unbegrenzte Anzahl an Operationen möglich und es kann fast wie auf unverschlüsselten Daten trainiert werden.

In [73]:
import tenseal as ts

poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 40]
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

seal_x_train = encryptX(X_train,ctx_training)
seal_y_train = encryptY(y_train,ctx_training)

Während dem Training wird also nach jedem Epoch die Gewichte dem Datenbesitzer zugesendet. Dieser ent- und verschlüsselt diese wieder mit den gleichen Parameter wie am Anfang des Trainings. Dieser Vorgang wird mit dem Aufruf der folgenden Methode simuliert.

In [74]:
def send_receive_weights(e_weights):
	weights = [item.decrypt() for item in e_weights]
	return np.array([ts.ckks_vector(ctx_training, label) for label in weights])

Der große Unterschied zum unverschlüsselten Training ist dabei, dass wir nicht aus der Ableitung der Gewichte ablesen können wie weit die Funktion noch optimiert werden muss. Dies liegt daran, da wir davon ausgehen, dass die Person, welche das Training ausführt, die entschlüsselten Gewichte erst zugesendet bekommt sobald das Training abgeschlossen ist.

In [75]:
def fit(x, y, epochs, learning_rate=0.00001):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])
	e_weights = np.array(encryptY(weights, ctx_training))

	for _ in range(epochs):
		y_pred = regression @ e_weights.T
		dw = regression.T @ (y - y_pred)
		e_weights = e_weights.T + (learning_rate * dw)
		e_weights = send_receive_weights(e_weights)

	return [item.decrypt() for item in e_weights]

In [79]:
e_weights = fit(seal_x_train, seal_y_train, 200, 0.0001)
e_weights = [e_weight[0] for e_weight in e_weights]

In [80]:
index = randint(0, len(X_test)-1)
print(f"""
Eingabe: {X_test[index]}
Wahres Ergebnis: {y_test[index]}
Ergebnis aus dem verschlüsselt trainierten Modell: {predict(e_weights, X_test[index])}
Ergebnis aus dem nicht verschlüsselt trainierten Modell: {predict(weights, X_test[index])}
""")


Eingabe: [-1.26418478 -0.13197948 -1.34022653 -1.18381211]
Wahres Ergebnis: 0
Ergebnis aus dem verschlüsselt trainierten Modell: -0.0015522208458345066
Ergebnis aus dem nicht verschlüsselt trainierten Modell: 0.04208134636702654



In [81]:
result = predict(e_weights, X_test)
meanError(y_test, result)

Mean Absolute Error: 0.17206092527074002
Mean Squared Error: 0.06821842140262571
Mean Root Squared Error: 0.2611865643608524


Wie zu sehen ist an den Ergebnissen hat das mit Hilfe der anderen Variante trainierte Modell eine höhere Genauigkeit. Dies wäre also ein Kompromiss zwischen der Genauigkeit und der Laufzeit, da das immer wieder aufs neue verschlüsseln der Gewichte sehr aufwändig ist.